# Practica 1B
Sergio Ramos Mesa\
David del Cerro Domínguez

### Ejercicio 1

Un grupo de 5 personas quiere cruzar un viejo y estrecho puente. Es una noche cerrada y se necesita llevar una linterna para cruzar. El grupo solo dispone de una linterna, a la que le quedan 5 minutos de batería.

1. Cada persona tarda en cruzar 10, 30, 60, 80 y 120 segundos, respectivamente.
2. El puente solo resiste un máximo de 2 personas cruzando a la vez, y cuando cruzan dos personas juntas, caminan a la velocidad del más lento.
3. No se puede lanzar la linterna de un extremo a otro del puente, así que cada vez que crucen dos personas, alguien tiene que volver a cruzar hacia atrás con la linterna a buscar a los compañeros que falten, y así hasta que hayan cruzado todos

In [1]:
 class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

In [440]:
class elPuente(Problem):
    
    def __init__(self, initial, goal = None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, (0, initial, (), 0), goal)
        self._actions = [(10,"10"),(30,"30"),(60,"60"),(80,"80"),(120,"120"),(30,"10","30"),(60,"10","60"),(80,"10","80"),(120,"10","120"),(60,"30","60"),(80,"30","80"),(120,"30","120"),(80,"60","80"),(120,"60","120"),(120,"80","120")]
        
    def actions(self, state):
        '''Devuelve las acciones validas para un estado.'''        
        t = 0
        for i in state[1]: t+= i
        for i in state[2]: t+= i    
      
        ret = list()
        #Recorre todas las acciones posibles
        for act in self._actions:
            i = 1  
            moves = list()
            actTime = 0

            #Controla que de tiempo a cruzar la barca
            while(i in range(len(act))):
                if(int(act[i]) > actTime): actTime = int(act[i])
                moves.append(int(act[i]))
                i+=1
                
            #Si supera el tiempo no hay posibilidad de seguir
            if((state[0] + actTime) < t):
                insert = True
                #Comprueba que las dos personas estén en el mismo lado del río
                for j in range(len(moves)):
                    if ((int(act[j]) in state[1] and state[3] == 0) or (int(act[j]) in state[2] and state[3] == 1)): 
                        insert = insert and True
                    else: insert = insert and False 
                    j+=1
                #Si cumple todas las condiciones se da como transición valida
                if (insert): ret.append(act) 
        return ret
    
    def result(self, state, act):
        '''Devuelve el estado resultante de aplicar una accion a un estado determinado.'''
        left = list(state[1])
        right = list(state[2])
        t = state[0] + act[0]

        i = 1        
        #Para todo el estado actual ejecuta el camino de ida y el de vuelta
        while(i in range(len(act))):
            aux = int(act[i])
            if state[3] == 0: #Ida
                left.remove(aux)
                right.append(aux)
                
            else: #Vuelta
                right.remove(aux)
                left.append(aux)
            i+=1
        turno = (state[3] + 1) % 2
        return (t, tuple(left), tuple(right), turno)
    
    def goal_test(self,estado):
        '''Devuelve si el estado actual es solución'''
        return (estado[0] <= 300) and (len(estado[1]) == 0) and (estado[3] == 1)

In [441]:
puente.initial

(0, (10, 30, 60, 80, 120), (), 0)

In [442]:
puente = elPuente((10,30,60,80,120))

In [443]:
puente.actions(puente.initial)

[(10, '10'),
 (30, '30'),
 (60, '60'),
 (80, '80'),
 (120, '120'),
 (30, '10', '30'),
 (60, '10', '60'),
 (80, '10', '80'),
 (120, '10', '120'),
 (60, '30', '60'),
 (80, '30', '80'),
 (120, '30', '120'),
 (80, '60', '80'),
 (120, '60', '120'),
 (120, '80', '120')]

In [444]:
puente.result(puente.initial, (120,"10","120"))

(120, (30, 60, 80), (10, 120), 1)

In [445]:
puente.goal_test(puente.initial)

False

In [451]:
puente.goal_test((300, (), (10,30,60,80,120), 1))

True

In [447]:
puente.actions(120,(30,60,80),(10.120),1)

TypeError: actions() takes 2 positional arguments but 5 were given

In [448]:
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

Para la ejecución se ha optado por la busqueda primero en anchura ya que la más eficiente dentro de las busquedas que no requieren de heurística. El tiempo de ejecución es ligeramente superior a otros algoritmos estudiados en clase y, aún así, son resultados muy buenos. Como podemos ver, se han adjuntado tanto los tiempo como las ejecuciones de todos los algorítmos comprobados.

In [462]:
breadth_first_tree_search(elPuente((10,30,60,80,120))).solution()

[(30, '10', '30'),
 (10, '10'),
 (60, '10', '60'),
 (10, '10'),
 (120, '80', '120'),
 (30, '30'),
 (30, '10', '30')]

568 ms ± 2.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [459]:
depth_first_tree_search(elPuente((10,30,60,80,120))).solution()

[(60, '10', '60'),
 (10, '10'),
 (30, '10', '30'),
 (30, '30'),
 (120, '80', '120'),
 (10, '10'),
 (30, '10', '30')]

998 ms ± 6.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [461]:
uniform_cost_search(elPuente((10,30,60,80,120))).solution()

[(30, '10', '30'),
 (10, '10'),
 (60, '10', '60'),
 (10, '10'),
 (120, '80', '120'),
 (30, '30'),
 (30, '10', '30')]

2.92 s ± 26.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [463]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# The solvability of a configuration can be checked by calculating the Inversion Permutation. If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        
        return inversion % 2 == 0   

In [467]:
def resuelve_puente_puzzle(estado_inicial, algoritmo, h=None):
    p = Problema_con_Analizados(elPuente(estado_inicial))
    if p.check_solvability(estado_inicial):
        if h: 
            sol = algoritmo(p,h).solution()
        else: 
            sol = algoritmo(p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [468]:
E1 = (10,30,60,80,120)
resuelve_puente_puzzle(E1,breadth_first_tree_search)

Solución: [(30, '10', '30'), (10, '10'), (60, '10', '60'), (10, '10'), (120, '80', '120'), (30, '30'), (30, '10', '30')]
Algoritmo: breadth_first_tree_search
Longitud de la solución: 7. Nodos analizados: 13117
